In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")

In [18]:
from pipeline.dataset.newsqa import load_news_qa_dataset
newsqa = load_news_qa_dataset()

Loading NewsQA dataset from /workspace/final_project/qa/newsqa.json


In [19]:
newsqa

[{'qid': 0,
  'docid': 20952,
  'question': '서울과 부산 구간의 1Gbps 양자암호통신 3년 계약 기준 월 요금은?',
  'answer': '3억8650만원'},
 {'qid': 1,
  'docid': 3648,
  'question': 'KF21 1호기가 하중보정·구조시험을 한 날과 장소는?',
  'answer': '지난 6일 구조시험동'},
 {'qid': 2,
  'docid': 819,
  'question': '엔시트론이 4일 투자 목적으로 취득한 HLB글로벌 신주인수권부사채의 양수금액은?',
  'answer': '50억원'},
 {'qid': 3,
  'docid': 24299,
  'question': '동국제강이 4일 매각한 DKSC 지분을 매수한 곳은 어디인가?',
  'answer': '중국 강음 지방 정부'},
 {'qid': 4,
  'docid': 9012,
  'question': '근로복지공단 부산의원은 언제 어디에 개원했나?',
  'answer': '5일 부산진구 범천동'},
 {'qid': 5,
  'docid': 8024,
  'question': '5일 국회 회의에서 류성걸 위원장이 밝힌 월세 세액공제 확대 상한은 몇 %인가?',
  'answer': '15%'},
 {'qid': 6,
  'docid': 7314,
  'question': '한컴은 언제까지 해외 상장을 목표로 했나?',
  'answer': '2024년까지'},
 {'qid': 7,
  'docid': 4572,
  'question': '체계종합기업 공고는 7월 말 어디에 게시될 예정인가?',
  'answer': '나라장터'},
 {'qid': 8,
  'docid': 24132,
  'question': '지난 3월 말 기준 KB·신한·우리·하나 4대 은행의 1년 이상 입출금 없는 예금 잔액은 얼마인가?',
  'answer': '15조7천676억원'},
 {'qid': 9,
  'docid': 3358,
 

In [67]:
from pipeline.qdrant.client import QdrantService

qdrant = QdrantService()

In [30]:
qdrant.get_collection_names()

['bge_500_200', 'bge_500_150', 'bge_500_50', 'bge_500_0', 'bge_500_100']

In [46]:
from pipeline.retriever.embed import load_encoding_model

model = load_encoding_model("dragonkue/bge-m3-ko")

Model is on device: cuda:0


In [65]:
from pipeline.retriever.embed import simple_encode

qid=0
embeddings = simple_encode(
    texts=[newsqa[qid]['question']],
    model=model, batch_size=32
)

print(newsqa[qid]['question'])
print(newsqa[qid]['answer'])
print(newsqa[qid]['docid'])

서울과 부산 구간의 1Gbps 양자암호통신 3년 계약 기준 월 요금은?
3억8650만원
20952


In [68]:
result = qdrant.search(collection_name=qdrant.get_collection_names()[0], query_vector=embeddings[0], limit=5)

In [69]:
result

[{'id': 20952003,
  'score': 0.77782804,
  'chunk_id': 3,
  'category': 'IT과학',
  'press': '디지털데일리 ',
  'link': 'https://n.news.naver.com/mnews/article/138/0002128281?sid=105',
  'bucket': 'train',
  'chunked_text': '7년 이상 장기 계약 시 이용료는 최소 4000만원으로 3년 계약으로 이용할 때보다 50% 이상 저렴한 비용으로 안정적인 양자암호통신 서비스를 이용할 수 있다. 장비 임대료는 최대 1Gbps 속도용 장비는 월 1650만원이며 10Gbps용은 1760만원이다. 예를 들어 서울과 용산 간 구간의 1Gbps 속도의 양자암호통신 서비스 이용료는 1억1650만원 3년 계약 기준 이며 서울과 부산 구간은 3억8650만원의 월 요금으로 이용할 수 있다.',
  'original_docid': 20952},
 {'id': 13307003,
  'score': 0.74774814,
  'chunk_id': 3,
  'category': 'IT과학',
  'press': '이코노미스트 ',
  'link': 'https://n.news.naver.com/mnews/article/243/0000028564?sid=105',
  'bucket': 'train',
  'chunked_text': '비용은 기존의 전용 회선 이용료 외에 양자채널이용료와 양자통신암호화장비 QENC 양자키관리장비 QKMS QKD의 장비 임대료가 부과된다. 양자채널이용료는 속도 구분 없이 거리에 따라 3년 계약 기준 월 1억 3억4000만원의 요금제로 구성된다. 7년 이상 장기 계약 시 이용료는 최소 4000만원으로 3년 계약으로 이용할 때보다 50% 이상 저렴한 비용으로 안정적인 양자암호통신 서비스를 이용할 수 있다. 예컨대 서울과 용산 간 구간의 1Gbps 속도의 양자암호통신 서비스 이용료는 1억1650만원 3년 계약 기준

In [62]:
nearest[0].id % 1000

3

In [63]:
nearest[0].payload

{'category': 'IT과학',
 'press': '디지털데일리 ',
 'link': 'https://n.news.naver.com/mnews/article/138/0002128281?sid=105',
 'bucket': 'train',
 'chunked_text': '7년 이상 장기 계약 시 이용료는 최소 4000만원으로 3년 계약으로 이용할 때보다 50% 이상 저렴한 비용으로 안정적인 양자암호통신 서비스를 이용할 수 있다. 장비 임대료는 최대 1Gbps 속도용 장비는 월 1650만원이며 10Gbps용은 1760만원이다. 예를 들어 서울과 용산 간 구간의 1Gbps 속도의 양자암호통신 서비스 이용료는 1억1650만원 3년 계약 기준 이며 서울과 부산 구간은 3억8650만원의 월 요금으로 이용할 수 있다.',
 'original_docid': 20952}

In [83]:
from pipeline.generator.prompter import PromptGenerator
from pipeline.generator.llm import OpenAIGenerator

generator = OpenAIGenerator(model_name="gpt-4o-mini")

In [71]:
newsqa[qid]['question']

'서울과 부산 구간의 1Gbps 양자암호통신 3년 계약 기준 월 요금은?'

In [77]:
[i['chunked_text'] for i in result]

['7년 이상 장기 계약 시 이용료는 최소 4000만원으로 3년 계약으로 이용할 때보다 50% 이상 저렴한 비용으로 안정적인 양자암호통신 서비스를 이용할 수 있다. 장비 임대료는 최대 1Gbps 속도용 장비는 월 1650만원이며 10Gbps용은 1760만원이다. 예를 들어 서울과 용산 간 구간의 1Gbps 속도의 양자암호통신 서비스 이용료는 1억1650만원 3년 계약 기준 이며 서울과 부산 구간은 3억8650만원의 월 요금으로 이용할 수 있다.',
 '비용은 기존의 전용 회선 이용료 외에 양자채널이용료와 양자통신암호화장비 QENC 양자키관리장비 QKMS QKD의 장비 임대료가 부과된다. 양자채널이용료는 속도 구분 없이 거리에 따라 3년 계약 기준 월 1억 3억4000만원의 요금제로 구성된다. 7년 이상 장기 계약 시 이용료는 최소 4000만원으로 3년 계약으로 이용할 때보다 50% 이상 저렴한 비용으로 안정적인 양자암호통신 서비스를 이용할 수 있다. 예컨대 서울과 용산 간 구간의 1Gbps 속도의 양자암호통신 서비스 이용료는 1억1650만원 3년 계약 기준 이며 서울과 부산 구간은 3억8650만원의 월 요금으로 이용할 수 있다. KT는 양자암호통신 기술 선도를 위해 고속 양자암호통신 시스템 개발과 서울 부산 최장거리 양자암호통신 서비스 실증 국제표준화 선도 등 연구개발을 활발히 진행 중이다.',
 '양자채널이용료는 속도 구분 없이 거리에 따라 3년 계약 기준 월 1억 3억4천만원의 요금제로 구성된다. 7년 이상 장기 계약 시 이용료는 최소 4천만원으로 3년 계약으로 이용할 때보다 50% 이상 저렴한 비용으로 안정적인 양자암호통신 서비스를 이용할 수 있다. 장비 임대료는 최대 1Gbps 속도용 장비는 월 1천650만원이며 10Gbps용은 1천760만원이다. 예컨대 서울과 용산 간 구간의 1Gbps 속도의 양자암호통신 서비스 이용료는 3년 계약 기준 1억1천650만원이며 서울과 부산 구간은 3억8천650만원의 월 요금으로 이용할 수 있다. KT는 대한민국의 양자암

In [81]:
prompts = PromptGenerator.generate_answer_with_docs(docs=[i['chunked_text'] for i in result], question=newsqa[qid]['question'])[0]

In [84]:
generation_response = generator.generate(prompts)

In [86]:
generation_response[0]

'```json\n{"answer": "서울과 부산 구간의 1Gbps 양자암호통신 3년 계약 기준 월 요금은 3억8650만원이다."}\n```'

In [87]:
print(newsqa[qid]['answer'])

3억8650만원


In [106]:
from pipeline.chain.generation import RAGChain
from pipeline.common import setup_logger

logger = setup_logger(name="RAGChainLogger")

ragchain = RAGChain(logger=logger)

Model is on device: cuda:0


In [107]:
result = ragchain.ask("현재 국토부 장관 이름은?")
print(result)

[('```json\n{"answer": "원희룡"}\n```', [{'id': 12694000, 'score': 0.5524032, 'original_chunk_id': 0, 'category': 'economy', 'press': '연합뉴스 ', 'link': 'https://n.news.naver.com/mnews/article/001/0013291336?sid=101', 'bucket': 'train', 'chunked_text': "원희룡 국토장관 유튜버 활동 재개\n세종 연합뉴스 김동규 기자 원희룡 국토교통부 장관이 현직 장관으로는 처음으로 유튜버로 활동한다. 원 장관은 5일 자신의 유튜브 채널 원희룡TV 에 국토부장관도 당할뻔한 신종 전세사기 수법 그 실체와 대책은 이라는 제목의 13분27초짜리 동영상을 올렸다.\n원희룡 국토교통부 장관이 현직 장관으로는 처음으로 유튜버로 활동하는 모습을 보이는 가운데 5일 자신의 유튜브 채널 원희룡TV 에'당할뻔한 신종 전세사기 수법 그 실체와 대책은'이라는 제목의 13분27초짜리 동영상을 올렸다.", 'original_docid': 12694}, {'id': 23542001, 'score': 0.5476997, 'original_chunk_id': 1, 'category': 'economy', 'press': '경향신문 ', 'link': 'https://n.news.naver.com/mnews/article/032/0003157764?sid=101', 'bucket': 'validation', 'chunked_text': '원희룡 국토교통부 장관 유튜브 화면 캡쳐 원희룡 국토교통부 장관이 우리나라 최초의 ‘유튜버’ 겸 국토부 장관이 됐다. 원 장관은 5일 자신의 유튜브 채널 원희룡TV 에 국토부장관도 당할뻔한 신종 전세사기 수법 그 실체와 대책은 이라는 제목의 13분짜리 영상을 게시했다. 원 장관은 영상에서 “지나치게 정치적 발언은 하지 않는다는 단서를 달고 국제정세 경제상황 생활콘텐츠와 관련해 활동할 수 있

In [109]:
result = ragchain.ask(["최근의 한중 관계는 어떠한가?", "최근의 미중 관계는 어떠한가?", "최근의 한일관계는 어떠한가?"])
print(result)

[('```json\n{"answer": "최근 한중 관계는 사드 사태 이후 더욱 악화되어 있으며, 중앙정부 간 관계가 어려운 상황에서 민간과 지자체 간의 협력이 필요하다는 목소리가 나오고 있다."}\n```', [{'id': 16651001, 'score': 0.5979711, 'original_chunk_id': 1, 'category': 'economy', 'press': '이데일리 ', 'link': 'https://n.news.naver.com/mnews/article/018/0005258813?sid=101', 'bucket': 'train', 'chunked_text': '한중 수교 30년 사드 사태 이후…악화일로 걷는 양국 尹정부 ‘한중 관계 재설정’ 최대 숙제 중 하나로 부상 기업과 中지자체간 협력·자매도시 간 우호활동 대안 전문가들 세밀한 관찰·관리가 중요…예방외교가 최선 이데일리 이준기 김상윤 기자 “대중 경제 의존도는 하루아침에 낮추기 어렵다.” 박승찬 용인대 중국학과 교수 “민간이 적극적으로 나서 한·중 관계의 새 돌파구를 찾는 건 바람직하다.” 김한권 국립외교원 교수 올해 수교 30주년을 맞은 한·중 관계가 새 국면을 맞았다. 미·중 간 패권경쟁에 따른 디커플링 탈동조화 속에 한·중 정부 간 간극이 더욱 커지면서 2016년 ‘사드 사태’ 이후 전례 없는 수준으로 ‘악화일로’를 걷고 있어서다. 그럼에도 ‘R 경기침체 의 공포’가 엄습한 상황에서 우리 수출의 4분의 1을 차지하는 중국을 포기하기란 쉽지 않다.', 'original_docid': 16651}, {'id': 16651000, 'score': 0.5883675, 'original_chunk_id': 0, 'category': 'economy', 'press': '이데일리 ', 'link': 'https://n.news.naver.com/mnews/article/018/0005258813?sid=101', 'bucket': 'train', 'chunked_text': '멀어진 韓

In [114]:
result[2]

('```json\n{"answer": "최근 한일 관계는 윤석열 대통령과 기시다 후미오 일본 총리의 회동을 통해 개선 여지가 보이고 있으며, 재계 주도의 경제 교류와 협력이 이어지고 있다. 그러나 양국 간 무역에서는 여전히 무역적자가 지속되고 있고, 과거의 긴장 상태에서도 회복 기미를 보이고 있다."}\n```',
 [{'id': 26757000,
   'score': 0.58107895,
   'original_chunk_id': 0,
   'category': 'economy',
   'press': '조선일보 ',
   'link': 'https://n.news.naver.com/mnews/article/023/0003701509?sid=101',
   'bucket': 'test',
   'chunked_text': '허창수 “코로나가 막바지인 것처럼 얼어붙은 한일 관계도…”',
   'original_docid': 26757},
  {'id': 13488007,
   'score': 0.576071,
   'original_chunk_id': 7,
   'category': 'economy',
   'press': '헤럴드경제 ',
   'link': 'https://n.news.naver.com/mnews/article/016/0002010604?sid=101',
   'bucket': 'train',
   'chunked_text': '윤석열 대통령과 기시다 후미오 일본 총리의 회동으로 한일 관계의 개선 여지가 넓어지고 재계 주도의 경제 교류·협력으로 ‘해빙무드’가 이어지고 있지만 정작 양국간 교역에서는 수교 이래 무역적자가 이어지고 있는 것으로 나타나 향후 교역 정상화가 이뤄질 경우 적자폭의 확대 가능성에 대해 관심이 모아진다.',
   'original_docid': 13488},
  {'id': 25212003,
   'score': 0.5697629,
   'original_chunk_id': 3,
   'category': 'econom